#### Andmete importimine

In [1]:
import pandas as pd
train = pd.read_csv('train_cleaner2.csv')

#### Kõik lahtrid tuleb viia numbrilisele kujule

In [2]:
pd.set_option("display.max_columns", None)
train.replace({False: 0, True: 1}, inplace=True) # Asendame TRUE ja FALSE 1 ja 0-ga

#Tühjad asendame keskmisega
train['Pressure'].fillna(3, inplace=True)
train['Satisfaction'].fillna(3, inplace=True)
train['Financial Stress'].fillna(3, inplace=True)

#CGPA puhul paneme tühjade asemele 0??
train['CGPA'].fillna(0, inplace=True)

train.head(10)


C:\Users\ksiim\AppData\Local\Temp\ipykernel_4676\3225228104.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.replace({False: 0, True: 1}, inplace=True) # Asendame TRUE ja FALSE 1 ja 0-ga
C:\Users\ksiim\AppData\Local\Temp\ipykernel_4676\3225228104.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original obje

,id,Name,Age,Pressure,CGPA,Satisfaction,Work/Study Hours,Financial Stress,Gender_Male,Working Professional or Student_Working Professional,Dietary Habits_Moderate,Dietary Habits_Unhealthy,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_Yes,Sleep Duration_7-8 hours,Sleep Duration_Less than 5 hours,Sleep Duration_More than 8 hours,City_Ahmedabad,City_Bangalore,City_Bhopal,City_Chennai,City_Delhi,City_Faridabad,City_Ghaziabad,City_Hyderabad,City_Indore,City_Jaipur,City_Kalyan,City_Kanpur,City_Kolkata,City_Lucknow,City_Ludhiana,City_Meerut,City_Mumbai,City_Nagpur,City_Nashik,City_Patna,City_Pune,City_Rajkot,City_Srinagar,City_Surat,City_Thane,City_Vadodara,City_Varanasi,City_Vasai-Virar,City_Visakhapatnam,Profession_Architect,Profession_Business Analyst,Profession_Chef,Profession_Chemist,Profession_Civil Engineer,Profession_Consultant,Profession_Content Writer,Profession_Customer Support,Profession_Data Scientist,Profession_Digital Marketer,Profession_Doctor,Profession_Educational Consultant,Profession_Electrician,Profession_Entrepreneur,Profession_Financial Analyst,Profession_Graphic Designer,Profession_HR Manager,Profession_Investment Banker,Profession_Judge,Profession_Lawyer,Profession_Manager,Profession_Marketing Manager,Profession_Mechanical Engineer,Profession_Pharmacist,Profession_Pilot,Profession_Plumber,Profession_Research Analyst,Profession_Researcher,Profession_Sales Executive,Profession_Software Engineer,Profession_Teacher,Profession_Travel Consultant,Profession_UX/UI Designer,Degree_Class 12,Degree_Master,Degree_PhD,y
0,0,Aaradhya,49.0,5.0,0.00,2.0,1.0,2.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Vivan,26.0,4.0,0.00,3.0,7.0,3.0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,2,Yuvraj,33.0,5.0,8.97,2.0,3.0,1.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,Yuvraj,22.0,5.0,0.00,1.0,10.0,1.0,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,4,Rhea,30.0,1.0,0.00,1.0,9.0,4.0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,Vani,59.0,2.0,0.00,5.0,7.0,5.0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6,6,Ritvik,47.0,5.0,0.00,2.0,6.0,2.0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,7,Rajveer,38.0,3.0,0.00,4.0,10.0,3.0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,8,Aishwarya,24.0,2.0,5.90,5.0,3.0,2.0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,Simran,42.0,4.0,0.00,1.0,7.0,2.0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


#### Nime veeru jätame välja ja eraldame X ja y edasiseks tegevuseks

In [3]:
X_train = train.drop(columns=['Name', 'y'])
y_train = train['y']
y_train.value_counts()

y
0    114955
1     25521
Name: count, dtype: int64

### Oversample ADASYN-iga
*ADASYN is similar to SMOTE but adds a little bit of random noise and adds more samples where the minority class is harder to distinguish.*

In [4]:
from imblearn.over_sampling import ADASYN
X_ADASYN, y_ADASYN = ADASYN().fit_resample(X_train, y_train)
y_ADASYN.value_counts()

y
1    115326
0    114955
Name: count, dtype: int64

### Random undersample

In [5]:
positive_cases = train[train['y'] == 1]
negative_sample = train[train['y'] == 0].sample(n=len(positive_cases), random_state=13)
train_undersampled = pd.concat([positive_cases, negative_sample], ignore_index=True)
train_undersampled.sort_index(inplace=True)

X_train_undersampled = train_undersampled.drop(columns=['Name', 'y'])
y_train_undersampled = train_undersampled['y']

Eraldame 85% ridadest treenimiseks ja 15% valideerimiseks:

In [6]:
from sklearn.model_selection import train_test_split
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train_undersampled, y_train_undersampled, train_size=0.85, random_state=36)

### Kirjutame treeningu ja validatsiooni tabelid csv-sse:

**NB! AJUTISELT AINULT 10000 RIDA, ET MUDEL KIIREMINI TOIMIKS**

In [7]:
#X_train_sub.drop(columns='id', inplace=True)
#X_val.drop(columns='id', inplace=True)

In [8]:
X_train_sub.to_csv('X_train_sub.csv')
y_train_sub.to_csv('y_train_sub.csv')
X_val.to_csv('X_val.csv')
y_val.to_csv('y_val.csv')